<a href="https://colab.research.google.com/github/nngiabao/MLPractice/blob/main/SpaceshipTitanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
#import libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#using KNN
from sklearn.impute import KNNImputer


In [82]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_test['Transported'] = False
df = pd.concat([df_train, df_test], sort = False)
df.drop(['Name'], axis = 1 ,inplace= True)
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


In [83]:
df.shape[0] == df_train.shape[0] + df_test.shape[0]

True

In [84]:
df.isna().sum()

,0
PassengerId,0
HomePlanet,288
CryoSleep,310
Cabin,299
Destination,274
Age,270
VIP,296
RoomService,263
FoodCourt,289
ShoppingMall,306


In [85]:
df[['Deck','Num','Side']] = df['Cabin'].str.split('/', expand=True)
df = df.drop(columns=['Cabin'])
df['Deck'] = df['Deck'].fillna('U')
df['Num'] = df['Num'].fillna(-1)
df['Side'] = df['Side'].fillna('U')


In [86]:
df['HomePlanet'].value_counts()

,count
HomePlanet,
Earth,6865
Europa,3133
Mars,2684


In [87]:
df['Deck'].value_counts()

,count
Deck,
F,4239
G,3781
E,1323
B,1141
C,1102
D,720
A,354
U,299
T,11


In [88]:
df['Side'].value_counts()

,count
Side,
S,6381
P,6290
U,299


In [89]:
df['Deck'] = df['Deck'].map({'G': 0, 'F': 1, 'E': 2, 'D': 3, 'C': 4, 'B': 5, 'A': 6, 'U': 7})
df['Side'] = df['Side'].map({'P': 0, 'S': 1})

In [90]:
impute_lis = ['Age','VIP','Num','CryoSleep','Side','Deck','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
rest = list(set(df.columns) - set(impute_lis))
df_rest = df[rest]
imp = KNNImputer()
df_imputed = imp.fit_transform(df[impute_lis])
df_imputed = pd.DataFrame(df_imputed, columns = impute_lis)
df = pd.concat([df_rest.reset_index(drop = True), df_imputed.reset_index(drop = True)], axis = 1)

In [96]:
df['HomePlanet'] = df['HomePlanet'].fillna('U')
df['Destination'] = df['Destination'].fillna('U')
category_colls = ['HomePlanet', 'Destination']

for col in category_colls:
    df = pd.concat([df, pd.get_dummies(df[col], prefix = col)], axis = 1)


In [92]:
#feature egineering
bill_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
df['TotalSpend'] = df[bill_cols].sum(axis=1)
df['std_amt_spent'] = df[bill_cols].std(axis=1)
df['avg_amt_spent'] = df[bill_cols].mean(axis=1)
df['max_amt_spent'] = df[bill_cols].max(axis=1)

In [99]:
df = df.drop(['HomePlanet', 'Destination'], axis=1)
df.corr()['Transported'].sort_values(ascending=False)

,Transported
Transported,1.000000
CryoSleep,0.324366
HomePlanet_Europa,0.131977
Destination_55 Cancri e,0.083625
Deck,0.079457
Side,0.074207
FoodCourt,0.034739
PassengerId,0.014628
HomePlanet_U,0.006403
HomePlanet_Mars,0.005643
